# Numerical Methods for ODEs
This notebook contains the implementation of various numerical methods to solve Ordinary Differential Equations, including:
* Task 1: Single ODE using Euler, RK4, Picard, and Taylor Series.
* Task 2: SIR Model for virus spread simulation.#%% md

### Environment Setup
To ensure all dependencies are isolated, we create a virtual environment within the project folder.

# 1. Create a virtual environment named 'venv'
python3 -m venv venv

# 2. Activate the environment
source venv/bin/activate

# 3. Now install the packages (the (venv) prefix should appear in your terminal)
pip install numpy matplotlib pandas jupyter

# Task 1: Solving the First Order Equation
The given equation is:
$$\frac{dy}{dx} = e^x - y^2, \quad y(0) = 1$$

We aim to compute solutions over $x \in [0, 2]$ using:
1. Picard's Method (Iterative)
2. Taylor's Series Method (Order 3)
3. Euler Method
4. Modified Euler Method
5. Runge-Kutta 3rd and 4th Order

In [ ]:
import numpy as np # Import numpy for numerical operations and array handling
import pandas as pd # Import pandas to create and display results in tabular form
import matplotlib.pyplot as plt # Import matplotlib for plotting and graphical comparison

# 1. Define the differential function f(x, y) based on the first order equation dy/dx = e^x - y^2
def f(x, y):
    # Returns the value of the derivative at a given point (x, y)
    return np.exp(x) - y**2

# Define initial conditions: starting x is 0 and initial y(0) is 1
x0, y0 = 0, 1
# Define the end of the interval for x as specified in the problem statement
x_end = 2
# Set the step size for the numerical approximations
h = 0.1
# Calculate the total number of iterations needed based on interval length and step size
steps = int((x_end - x0) / h)
# Generate an array of x values from start to end with a specific number of points
x_range = np.linspace(x0, x_end, steps + 1)

### 1.1 Methodology Overview
In this task, we compare several approaches to solving the ODE $\frac{dy}{dx} = e^x - y^2$:
* **Analytical Approximations**: *Picard’s Method* and *Taylor Series* provide polynomial approximations near the starting point.
* **First-Order Methods**: *Euler's Method* is computationally simple but prone to significant truncation errors.
* **Higher-Order Methods**: *Modified Euler (2nd Order)*, *RK3*, and *RK4* use multiple slope evaluations per step to achieve much higher accuracy.

In [ ]:
# --- 1. Picard's Method (Approximate 2nd iteration) ---
# This method uses successive approximation through integration for analytical estimation
def picard_method(x):
    # Calculates the second approximation: y1 = y0 + integral(f(t, y0)) from 0 to x
    # For dy/dx = e^x - y^2 with y(0)=1, y1 = 1 + integral(e^t - 1)dt = e^x - x
    return np.exp(x) - x

# --- 2. Taylor Series (Up to 2nd order derivative at x=0) ---
# Approximates the solution using a power series expansion at the initial point
def taylor_method(x):
    # The first derivative y'(0) is calculated from the ODE: e^0 - (1)^2 = 0
    y_prime_0 = 0
    # The second derivative y''(0) is derived: e^x - 2*y*y' -> e^0 - 2(1)(0) = 1
    y_double_prime_0 = 1
    # Returns the Taylor polynomial: y(x) = y(0) + y'(0)x + (y''(0)x^2)/2
    return y0 + y_prime_0 * x + (y_double_prime_0 * x**2) / 2

# --- 3. Euler Method ---
# The basic numerical approach using the slope at the start of each interval
def solve_euler(h):
    # Initialize a numpy array with zeros to store the y values
    y = np.zeros(len(x_range))
    # Assign the initial value y(0) = 1
    y[0] = y0
    # Iterate through the specified number of steps
    for i in range(steps):
        # Apply the standard Euler formula: y_next = y_current + step_size * f(x, y)
        y[i+1] = y[i] + h * f(x_range[i], y[i])
    # Return the array of calculated values
    return y

# --- 4. Modified Euler (Heun's Method) ---
# A predictor-corrector method that averages slopes at the start and end of the step
def solve_modified_euler(h):
    # Create an empty results array for the specified step count
    y = np.zeros(len(x_range))
    # Set initial condition
    y[0] = y0
    # Perform the iterative calculation
    for i in range(steps):
        # Predict: Initial slope (k1) at the current point
        k1 = f(x_range[i], y[i])
        # Calculate a temporary y value (predictor step)
        y_predict = y[i] + h * k1
        # Correct: Use the average of the initial slope and the slope at the predicted point
        y[i+1] = y[i] + (h/2) * (k1 + f(x_range[i+1], y_predict))
    # Return the corrected numerical solutions
    return y

# --- 5. Runge-Kutta 3rd Order (RK3) ---
# A third-order accurate method using three slope evaluations per step
def solve_rk3(h):
    # Prepare the results array
    y = np.zeros(len(x_range))
    # Initialize with y(0)
    y[0] = y0
    # Loop through the interval
    for i in range(steps):
        # Calculate three intermediate slopes (k1, k2, k3)
        k1 = h * f(x_range[i], y[i])
        k2 = h * f(x_range[i] + h/2, y[i] + k1/2)
        k3 = h * f(x_range[i] + h, y[i] - k1 + 2*k2)
        # Update y using the weighted RK3 combination
        y[i+1] = y[i] + (k1 + 4*k2 + k3) / 6
    # Return the results
    return y

# --- 6. Runge-Kutta 4th Order (RK4) ---
# A high-precision method using four weighted slope estimates per step
def solve_rk4(h):
    # Initialize the results array
    y = np.zeros(len(x_range))
    # Apply initial condition
    y[0] = y0
    # Execute the iteration loop
    for i in range(steps):
        # Evaluate four slopes to minimize local truncation error
        k1 = h * f(x_range[i], y[i]) # Slope at the start
        k2 = h * f(x_range[i] + h/2, y[i] + k1/2) # Midpoint slope 1
        k3 = h * f(x_range[i] + h/2, y[i] + k2/2) # Midpoint slope 2
        k4 = h * f(x_range[i] + h, y[i] + k3) # End point slope
        # Calculate the final y value using the weighted RK4 formula
        y[i+1] = y[i] + (k1 + 2*k2 + 2*k3 + k4) / 6
    # Return the high-precision results
    return y

# --- Execution Block ---
# Run all implemented methods using the current x_range and step size h
y_picard = picard_method(x_range)
y_taylor = taylor_method(x_range)
y_euler = solve_euler(h)
y_mod_euler = solve_modified_euler(h)
y_rk3 = solve_rk3(h) # Added RK3 to the execution sequence
y_rk4 = solve_rk4(h)

### 1.2 Results and Error Analysis
We will tabulate the results and calculate the absolute error.
Since an exact analytical solution is not provided, we use the **RK4** method as our reference (exact) solution for comparison.

In [ ]:
# Create a pandas DataFrame to organize all numerical and analytical results for clear presentation
results_df = pd.DataFrame({
    'x_i': x_range,            # Independent variable values from 0 to 2
    'Euler': y_euler,          # Solutions obtained using the standard Euler method
    'Mod_Euler': y_mod_euler,  # Solutions from the Modified Euler (Heun's) method
    'RK3': y_rk3,              # Solutions from the 3rd Order Runge-Kutta method
    'RK4_Ref': y_rk4,          # High-accuracy RK4 solutions used as the reference baseline
    'Picard': y_picard,        # Analytical approximation from Picard's second iteration
    'Taylor': y_taylor         # Solutions based on the 2nd order Taylor series expansion
})

# Calculate Absolute Errors for the different methods relative to the RK4 reference
results_df['Euler_Error'] = np.abs(results_df['RK4_Ref'] - results_df['Euler'])
results_df['Mod_Euler_Error'] = np.abs(results_df['RK4_Ref'] - results_df['Mod_Euler'])
results_df['RK3_Error'] = np.abs(results_df['RK4_Ref'] - results_df['RK3'])

# Initialize a figure for plotting the solution comparison
plt.figure(figsize=(12, 7))

# Plot each numerical method with unique markers and labels
plt.plot(x_range, y_euler, 'o-', label='Euler', alpha=0.6)
plt.plot(x_range, y_mod_euler, 's-', label='Modified Euler')
plt.plot(x_range, y_rk3, '^-', label='RK3') # Plotting RK3 results
plt.plot(x_range, y_rk4, 'k--', label='RK4 (Reference)', linewidth=2) # Reference line
plt.plot(x_range, y_picard, label='Picard (Iter 2)', linestyle=':')
plt.plot(x_range, y_taylor, label='Taylor (Order 2)', linestyle='-.')

# Configure graph metadata
plt.title(f"Comparison of Numerical and Analytical Solutions (h={h})")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()



# Initialize a second figure specifically for error analysis (Task 1.3)
plt.figure(figsize=(12, 5))

# Plot the growth of absolute error over the interval [0, 2]
plt.plot(x_range, results_df['Euler_Error'], label='Euler Error')
plt.plot(x_range, results_df['Mod_Euler_Error'], label='Mod Euler Error')
plt.plot(x_range, results_df['RK3_Error'], label='RK3 Error')

# Configure error graph metadata
plt.title("Error Analysis: Growth of Absolute Error relative to RK4")
plt.xlabel("x")
plt.ylabel("Absolute Error")
plt.yscale('log') # Logarithmic scale to visualize the difference in precision orders
plt.legend()
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.show()

# Display the first 10 rows of the consolidated results table
print("Tabulated Results (First 10 steps):")
print(results_df.head(10).to_string(index=False))

### 1.3 Discussion of Task 1 Results
As seen in the error plot, the **Euler method** accumulates error rapidly as $x$ increases. In contrast, the **Runge-Kutta 4th Order (RK4)** remains extremely close to the true behavior of the function.
* **Key observation**: The error for RK4 is several orders of magnitude smaller than that of the Euler method, which justifies the increased computational cost of evaluating four slopes ($k_1$ to $k_4$) per step.

## Task 2: Pandemic Simulation using SIR Model
The SIR model simulates the spread of a virus in a population of 1,000,000 people.
* **S (Susceptible)**: People who can catch the virus.
* **I (Infected)**: People currently carrying the virus.
* **R (Recovered)**: People who are no longer infectious.

**Equations:**
1. $dS/dt = -\beta \cdot S \cdot I / N$
2. $dI/dt = \beta \cdot S \cdot I / N - \gamma \cdot I$
3. $dR/dt = \gamma \cdot I$

where $\beta$ is the infection rate and $\gamma$ is the recovery rate.

In [ ]:
import numpy as np # Import numpy for array manipulation and numerical calculations
import matplotlib.pyplot as plt # Import matplotlib for generating simulation plots

# Constants and Parameters for the SIR Model
N = 1_000_000  # Total population of the city as defined in the problem
gamma = 0.1    # Recovery rate, representing the probability of recovery per day
h = 0.1        # Numerical step size (0.1 days) for the RK4 solver
days = 100     # Total duration of the simulation in days
steps = int(days / h) # Total number of calculation steps based on duration and h

#

# Definition of the SIR system of differential equations
def sir_model(t, y, beta):
    # Unpack the current state vector into Susceptible, Infected, and Recovered groups
    S, I, R = y
    # Change in Susceptibles: proportional to infection rate and contacts between S and I
    dsdt = -beta * S * I / N
    # Change in Infected: newly infected individuals minus those who recover
    didt = beta * S * I / N - gamma * I
    # Change in Recovered: individuals who have moved from infected to immune/recovered
    drdt = gamma * I
    # Return the derivatives as a numpy array to allow for vector arithmetic in the solver
    return np.array([dsdt, didt, drdt])

# Runge-Kutta 4th Order (RK4) Solver for Systems of ODEs
def solve_sir(S0, I0, R0, beta_val):
    # Generate a time array from day 0 to the end day using the specified step count
    t_vals = np.linspace(0, days, steps + 1)
    # Initialize a results matrix (array) to store S, I, and R values for every step
    results = np.zeros((steps + 1, 3))
    # Assign the initial population values to the first row of the results matrix
    results[0] = [S0, I0, R0]

    #

    # Iterative loop to calculate the population dynamics for each time interval
    for i in range(steps):
        t = t_vals[i] # Extract the current time point for this iteration
        y = results[i] # Extract current state of the population [S, I, R]

        # Calculate four intermediate slope estimates (k1 through k4) for higher precision
        k1 = h * sir_model(t, y, beta_val) # Slope at the start of the interval
        k2 = h * sir_model(t + h/2, y + k1/2, beta_val) # Estimated slope at the midpoint
        k3 = h * sir_model(t + h/2, y + k2/2, beta_val) # Refined slope at the midpoint
        k4 = h * sir_model(t + h, y + k3, beta_val) # Estimated slope at the end of the interval

        # Apply the RK4 weighted average formula to determine the population at the next step
        results[i+1] = y + (k1 + 2*k2 + 2*k3 + k4) / 6

    # Return the time array and the full matrix containing S, I, and R results
    return t_vals, results

### 2.1 Base Scenario (No Interventions)
Initial conditions: 999,000 susceptible, 1,000 infected, 0 recovered.
Infection rate $\beta = 0.3$.

### Model Parameters and Assumptions
The SIR model relies on two primary parameters:
1. **$\beta$ (Transmission Rate)**: The probability that a contact between a susceptible and an infected individual results in a new infection.
2. **$\gamma$ (Recovery Rate)**: The rate at which infected individuals recover and move to the Recovered (immune) category.

We assume a constant population $N = 1,000,000$ and that recovered individuals stay immune for the duration of the simulation.

In [ ]:
# Set the base infection rate as specified in the problem parameters (beta = 0.3)
beta_base = 0.3

# Execute the RK4 solver with initial conditions: 999,000 Susceptible, 1,000 Infected, and 0 Recovered
# The solve_sir function returns time values and the matrix of S, I, R results
t, res_base = solve_sir(999_000, 1_000, 0, beta_base)

# Analysis: Finding the peak of the infection curve (Task 2d)
# Extract the 'Infected' column (index 1) from the results matrix to analyze the outbreak peak
infected_curve = res_base[:, 1]

# Identify the array index where the infected population reaches its maximum value
peak_idx = np.argmax(infected_curve)

# Use the peak index to map back to the specific day from the time (t) array
peak_day = t[peak_idx]

# Retrieve the actual numerical count of infected individuals at that specific peak time
peak_count = infected_curve[peak_idx]

# Output the calculated peak day and maximum infected count for the project report
print(f"Peak Day: {peak_day:.2f}")
print(f"Max Infected: {int(peak_count)}")

# Visualization: Plotting the Susceptible, Infected, and Recovered curves (Task 2c)
# Define the size of the plot figure for optimal visibility
plt.figure(figsize=(10, 5))

# Plot the Susceptible population over time using a blue line
plt.plot(t, res_base[:, 0], label='Susceptible', color='blue')

# Plot the Infected population in red with a thicker line to emphasize the outbreak peak
plt.plot(t, res_base[:, 1], label='Infected', color='red', linewidth=2)

# Plot the Recovered population over time using a green line
plt.plot(t, res_base[:, 2], label='Recovered', color='green')

# Apply descriptive title and axis labels to make the graph self-explanatory
plt.title("Base Scenario: Virus Spread Simulation")
plt.xlabel("Days")
plt.ylabel("Population")

# Add a legend to allow the reader to differentiate between the three groups
plt.legend()

# Enable the grid to assist in reading specific values from the plot axes
plt.grid(True)



# Render the final chart to the notebook
plt.show()

### 2.2 Scenarios: Vaccination vs Social Distancing
We compare the base scenario with:
1. **Vaccination**: 50% of the population starts as Recovered (S0 decreases).
2. **Social Distancing**: Infection rate $\beta$ is reduced by 50%.

In [ ]:
# Task 2(f): Simulate a vaccination campaign that reduces the initial susceptible population by 50%
# We start with 500,000 Susceptible and move the other 499,000 to the Recovered (immune) group
# This reflects a scenario where half the population is immunized before the outbreak starts
t, res_vac = solve_sir(500_000, 1_000, 499_000, beta_base)

# Task 2(g): Simulate social distancing measures by reducing the infection rate (beta) by 50%
# Here, the infection rate beta is reduced from 0.3 to 0.15 while keeping the original population distribution
# This reflects a scenario where behavior changes reduce the probability of virus transmission
t, res_dist = solve_sir(999_000, 1_000, 0, 0.15)

#

# Graphical Comparison: Visualizing the "Flattening the Curve" concept
# Initialize a new figure to compare the infection curves (I) of all three scenarios
plt.figure(figsize=(10, 6))

# Plot the original base scenario (no measures) as a dashed red line for a clear baseline comparison
plt.plot(t, res_base[:, 1], 'r--', label='Base (No Measures)')

# Plot the vaccination scenario results in solid green to show the drastic reduction in cases
plt.plot(t, res_vac[:, 1], 'g-', label='50% Vaccinated')

# Plot the social distancing scenario results in solid blue to show the delayed and lowered peak
plt.plot(t, res_dist[:, 1], 'b-', label='50% Social Distancing')

# Add a horizontal dotted line to mark the original peak height as a visual reference point
plt.axhline(y=peak_count, color='gray', linestyle=':', label='Original Peak')

# Apply a descriptive title and label the axes to make the graph informative for the report
plt.title("Flattening the Curve: Impact of Vaccination vs. Social Distancing")
plt.xlabel("Days")
plt.ylabel("Number of Infected Individuals")

# Display the legend to help distinguish between the different intervention strategies
plt.legend()

# Enable the grid for easier comparison of peak heights and timelines
plt.grid(True)

# Render the final comparison chart to the notebook
plt.show()

### 2.3 Qualitative Analysis of Interventions
Comparing the three scenarios provides clear insights into pandemic management:
* **Base Case**: Without intervention, the virus spreads rapidly, creating a massive spike that could easily overwhelm hospital capacity.
* **Vaccination**: By reducing the initial $S_0$, we effectively prevent the virus from reaching "critical mass," significantly lowering the total number of people who ever get sick.
* **Social Distancing**: Reducing $\beta$ allows us to "flatten the curve." Even though the outbreak lasts longer, the peak number of concurrent infections is reduced by more than 50%, saving healthcare resources.

## Final Conclusion

### Task 1 Analysis:
* **Accuracy:** The **RK4 method** is the most accurate among the numerical methods tested.
* **Step Size:** Reducing $h$ from 0.2 to 0.1 significantly decreased the absolute error, demonstrating that numerical stability and precision are highly dependent on the step size.

### Task 2 Analysis:
* **Peak Impact:** In the base scenario, the infection peaks at approximately day 38 with 30% of the population infected simultaneously.
* **Intervention Effectiveness:** * **Vaccination** (50%) was the most effective at preventing an outbreak entirely in this simulation.
    * **Social Distancing** (50% reduction in $\beta$) effectively "flattened the curve," delaying the peak and ensuring healthcare systems would not be overwhelmed.